In [1]:
from lxml import etree

In [2]:
from typing import List, Tuple

In [3]:
def load_sentirueval_2016(file_name: str) -> Tuple[List[str], List[str]]:
    texts = []
    labels = []
    with open(file_name, mode='rb') as fp:
        xml_data = fp.read()
    root = etree.fromstring(xml_data)
    for database in root.getchildren():
        if database.tag == 'database':
            for table in database.getchildren():
                if table.tag != 'table':
                    continue
                new_text = None
                new_label = None
                for column in table.getchildren():
                    if column.get('name') == 'text':
                        new_text = str(column.text).strip()
                        if new_label is not None:
                            break
                    elif column.get('name') not in {'id', 'twitid', 'date'}:
                        if new_label is None:
                            label_candidate = str(column.text).strip()
                            if label_candidate in {'0', '1', '-1'}:
                                new_label = 'negative' if label_candidate == '-1' else \
                                    ('positive' if label_candidate == '1' else 'neutral')
                                if new_text is not None:
                                    break
                if (new_text is None) or (new_label is None):
                    raise ValueError('File `{0}` contains some error!'.format(file_name))
                texts.append(new_text)
                labels.append(new_label)
            break
    return texts, labels

In [4]:
texts, labels = load_sentirueval_2016('bank_train_2016.xml')

In [5]:
print('Number of texts is {0}, number of labels is {1}.'.format(len(texts), len(labels)))

Number of texts is 9392, number of labels is 9392.


In [6]:
#1)токенизацию с учётом возможных смайлов (базовая токенизация некорректно
#работает со знаками препинания и прочими неалфавитными и нецифровыми
#символами, из которых как раз и могут состоять тонально значимые смайлы)

In [7]:
# 2) лемматизацию с учётом контекста, чтобы успешно разрешать морфоомонимию вида
# “​ мы ​ стали лучше программировать​ ” - “​ мы выплавляем больше ​ стали​ ” (для такой
# лемматиции можно использовать, например, библиотеку ​ UDPipe или её адаптацию
# под ​ SpaCy​ )

In [8]:
# 3) удаление стоп-слов по словарям и/или правилам (например, описанным в виде
# регулярных выражений).

In [9]:
from nltk.tokenize import TweetTokenizer
from rnnmorph.predictor import RNNMorphPredictor

predictor = RNNMorphPredictor(language='ru')
tokenizer = TweetTokenizer()

def drop_junk(s):
    return ' '.join([word for word in tokenizer.tokenize(s) if word.isalpha() or '!' in word or '?' in word \
                     or '(' in word or ')' in word])

def lemmatization(s):
    toks = tokenizer.tokenize(s)
    forms = predictor.predict(toks)
    return ' '.join([f.normal_form for f in forms])

texts, labels = load_sentirueval_2016('bank_train_2016.xml')

texts = list(map(drop_junk, texts))
texts = list(map(lemmatization, texts))

In [10]:
from stop_words import get_stop_words

stop_words = get_stop_words('ru')

In [11]:
import random

In [12]:
for idx in random.choices(list(range(len(texts))), k=20):
    print('{0} => {1}'.format(labels[idx], texts[idx]))

neutral => сбербанк не в список ?
neutral => заявка на кредит в втб
neutral => потребительский в тюмень втб
negative => у я такой история с длиться месяц и длиться бы ещё если бы про он не написать газета ведомость
neutral => denizbank сбербанк покупать в турция бизнес по обслуживание физлицо
neutral => кредит в райффайзен банк
neutral => кредит на личный нужда сбербанк
neutral => онлайн анкета на кредит банк москва
neutral => кредит райффайзенбанк физический лицо
neutral => как взять кредит за день в альфа банк
neutral => сбербанк в челябинск отделение
neutral => как взять потребительский кредит в альфа банк
neutral => перекредитование кредит втб
neutral => сбербанк капитал вместе с миноритарий выставить на торг процент акция компания премиум спир
neutral => делать ли распечатка операция карта сбербанк
neutral => райффайзенбанк потребительский кредит нижний новгород
neutral => в альфа банк кредит для студент
negative => полиция задержать грабитель укравший из московский филиал сбербан

In [13]:
positive_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'positive']
negative_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'negative']
neutral_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'neutral']

In [14]:
for cur in positive_tweets[:5]: print(cur)

главный чтоб банк сбер и втб ! ! !
самый выгодный автокредит в втб
легко можно получить денежный кредит ы втб банка
снижение процентный ставка по кредит на недвижимость сбербанк
в наш сбербанк прийти американец и попросить сфоткаться с он у банкомат я ващий быть в шок и на фотка полюбома получиться смешной xd


In [15]:
for cur in negative_tweets[:5]: print(cur)

обязательно про сбербанк написать ! временами хлёсткий интернет магазин финт выкручивать )
втб и сбер точно операция или запрет на кредит ?
канада ввести санкция против газпромбанк вэба новатэк и российский оборонный компания
rt если сбер и втб попасть под санкция быть жопа
rt ввести санкция против оборонный и сырьевой компания и банк среди он в частность


In [16]:
for cur in neutral_tweets[:5]: print(cur)

взять кредит тюмень альфа банк
мнение о кредитный карта втб
райффайзенбанк снижение ключевой ставка цб на заседание в этот пятница очень маловероятный
современный состояние кредитный поведение в россия сбербанк
оформить краткосрочный кредит оао банк москва


In [17]:
from nltk import word_tokenize

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
vectorizer = CountVectorizer(lowercase=True, tokenizer=word_tokenize)

In [20]:
vectorizer.fit(texts)

C:\Users\PC\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(tokenizer=<function word_tokenize at 0x00000279F3368820>)

In [21]:
print(vectorizer.get_feature_names()[0:20])

['!', '(', ')', '-', '8', ':', ';', '=', '?', 'a', 'abncapital', 'admitad', 'ag', 'agro', 'airlines', 'ajhvf', 'alfa', 'alfabank', 'alfaclick', 'aliexpress']


In [22]:
print(len(vectorizer.get_feature_names()))

6660


In [23]:
X = vectorizer.transform(texts)

In [24]:
print(X)

  (0, 419)	1
  (0, 555)	1
  (0, 852)	1
  (0, 2494)	1
  (0, 5979)	1
  (1, 1081)	1
  (1, 2244)	1
  (1, 2498)	1
  (1, 2889)	1
  (1, 3358)	1
  (2, 764)	1
  (2, 1881)	1
  (2, 2323)	1
  (2, 2744)	1
  (2, 2998)	1
  (2, 3791)	1
  (2, 4764)	1
  (2, 4860)	1
  (2, 5410)	1
  (2, 5624)	1
  (2, 6385)	1
  (2, 6601)	1
  (3, 764)	1
  (3, 2498)	1
  (3, 4015)	1
  :	:
  (9389, 4905)	1
  (9389, 5106)	1
  (9389, 5461)	1
  (9389, 6134)	1
  (9389, 6277)	1
  (9389, 6599)	1
  (9390, 764)	1
  (9390, 3320)	1
  (9390, 3386)	1
  (9390, 4069)	1
  (9390, 4317)	1
  (9390, 4471)	1
  (9390, 5056)	1
  (9390, 5157)	1
  (9390, 5207)	1
  (9390, 6261)	1
  (9391, 764)	1
  (9391, 1970)	1
  (9391, 2494)	1
  (9391, 2777)	1
  (9391, 3320)	1
  (9391, 4686)	1
  (9391, 5056)	1
  (9391, 5124)	1
  (9391, 6048)	1


In [25]:
print(texts[0])

взять кредит тюмень альфа банк


In [26]:
print(X[0])

  (0, 419)	1
  (0, 555)	1
  (0, 852)	1
  (0, 2494)	1
  (0, 5979)	1


In [27]:
print(vectorizer.get_feature_names()[6321])

хакамада


In [28]:
print(vectorizer.get_feature_names()[5979])

тюмень


In [29]:
from sklearn.feature_extraction.text import TfidfTransformer

In [30]:
transformer = TfidfTransformer().fit_transform(X)

In [31]:
#X_transformed = transformer.transform(X)

In [32]:
print(transformer[0])

  (0, 5979)	0.7391351947070658
  (0, 2494)	0.22539383399193713
  (0, 852)	0.4219051594577849
  (0, 555)	0.26376023593941833
  (0, 419)	0.3940854703028856


In [33]:
transformer1 = TfidfTransformer().fit(X)

In [34]:
X_transformed = transformer1.transform(X)

In [35]:
print(X_transformed[0])

  (0, 5979)	0.7391351947070658
  (0, 2494)	0.22539383399193713
  (0, 852)	0.4219051594577849
  (0, 555)	0.26376023593941833
  (0, 419)	0.3940854703028856


In [36]:
print(vectorizer.get_feature_names()[6321])

хакамада


In [37]:
print(vectorizer.get_feature_names()[5979])

тюмень


In [38]:
tokens_with_IDF = list(zip(vectorizer.get_feature_names(), transformer1.idf_))

In [39]:
for feature, idf in tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

4.417620 => !
4.805386 => (
4.443938 => )
8.355961 => -
9.049108 => 8
6.020586 => :
8.068278 => ;
8.201810 => =
4.440610 => ?
9.049108 => a
9.454573 => abncapital
9.454573 => admitad
9.049108 => ag
9.049108 => agro
9.454573 => airlines
9.454573 => ajhvf
9.049108 => alfa
8.355961 => alfabank
9.049108 => alfaclick
9.454573 => aliexpress


In [40]:
sorted_tokens_with_IDF = sorted(tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [41]:
for feature, idf in sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => abncapital
9.454573 => admitad
9.454573 => airlines
9.454573 => ajhvf
9.454573 => aliexpress
9.454573 => api
9.454573 => app
9.454573 => apps
9.454573 => appstore
9.454573 => autopaymts
9.454573 => back
9.454573 => bankvtb
9.454573 => best
9.454573 => biznews
9.454573 => blackberry
9.454573 => blatt
9.454573 => bump
9.454573 => camp
9.454573 => cash
9.454573 => cetelem


In [42]:
from sklearn.feature_selection import SelectPercentile, chi2

In [43]:
selector = SelectPercentile(chi2, percentile=20)

In [44]:
selector.fit(X_transformed, labels)

SelectPercentile(percentile=20,
                 score_func=<function chi2 at 0x0000027986ECFC10>)

In [45]:
selected_tokens_with_IDF = [tokens_with_IDF[idx] for idx in selector.get_support(indices=True)]

In [46]:
print(len(selected_tokens_with_IDF))

1332


In [47]:
for feature, idf in selected_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

4.417620 => !
4.805386 => (
4.443938 => )
6.020586 => :
8.201810 => =
4.440610 => ?
9.049108 => apple
9.454573 => appstore
8.761426 => awards
8.538282 => boerse
9.049108 => brkng
9.049108 => cemitsvetic
9.454573 => come
9.454573 => euro
7.056678 => finance
7.439670 => fitch
8.761426 => ft
7.845135 => ftse
7.151988 => global
9.454573 => hsbc


In [48]:
selected_and_sorted_tokens_with_IDF = sorted(selected_tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [49]:
for feature, idf in selected_and_sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => appstore
9.454573 => come
9.454573 => euro
9.454573 => hsbc
9.454573 => london
9.454573 => moneynews
9.454573 => n
9.454573 => nalexandrowa
9.454573 => play
9.454573 => plc
9.454573 => rdx
9.454573 => softkey
9.454573 => spo
9.454573 => автолюбитель
9.454573 => автоматический
9.454573 => автопроверка
9.454573 => адекватный
9.454573 => азс
9.454573 => ай
9.454573 => аккредитовый
